# Laboratorio 5: Redes Neuronales Recurrentes (RNN) para Series de Tiempo

## Autores:

- Nelson García 22434
- Christian Echeverría 221



## Carga y exploración de datos:



Importar las librerías:

In [9]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import EarlyStopping

Cargamos los datos

In [10]:
df = pd.read_csv('./Datos/IPN31152N.csv')
df.head()

,observation_date,IPN31152N
0,1972-01-01,60.1519
1,1972-02-01,67.2727
2,1972-03-01,74.4700
3,1972-04-01,78.3594
4,1972-05-01,85.0321


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 642 entries, 0 to 641
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   observation_date  642 non-null    object 
 1   IPN31152N         642 non-null    float64
dtypes: float64(1), object(1)
memory usage: 10.2+ KB


Se convierte la fecha de Object a DateTime

In [12]:
df = pd.read_csv('./Datos/IPN31152N.csv',
                 index_col = 'observation_date',
                 parse_dates = True)


In [13]:
df.head(20)


,IPN31152N
observation_date,
1972-01-01,60.1519
1972-02-01,67.2727
1972-03-01,74.4700
1972-04-01,78.3594
1972-05-01,85.0321
1972-06-01,100.9147
1972-07-01,100.4435
1972-08-01,96.6660
1972-09-01,86.0726


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 642 entries, 1972-01-01 to 2025-06-01
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   IPN31152N  642 non-null    float64
dtypes: float64(1)
memory usage: 10.0 KB


In [15]:
fig = px.line(df)
fig.show()

## División de conjuntos

In [16]:
len(df)

642

Vamos a tomar los últimos 24 meses para nuestro set de tests.

In [17]:
tamanio_prueba = 24

In [18]:
indice_prueba = len(df) - tamanio_prueba

In [19]:
entreno = df.iloc[: indice_prueba]
prueba = df.iloc[indice_prueba:]

In [20]:
entreno

,IPN31152N
observation_date,
1972-01-01,60.1519
1972-02-01,67.2727
1972-03-01,74.4700
1972-04-01,78.3594
1972-05-01,85.0321
...,...
2023-02-01,119.2751
2023-03-01,122.0175
2023-04-01,126.6790


In [21]:
prueba

,IPN31152N
observation_date,
2023-07-01,126.0569
2023-08-01,122.6748
2023-09-01,116.2177
2023-10-01,109.3279
2023-11-01,89.3353
2023-12-01,85.2839
2024-01-01,85.9573
2024-02-01,110.0548
2024-03-01,124.9787


## Normalización y preparación

In [22]:
scaler = MinMaxScaler()

In [23]:
scaler.fit(entreno)

MinMaxScaler()

In [24]:
datos_entreno_escalados = scaler.transform(entreno)
datos_prueba_escalados = scaler.transform(prueba)

## Generador de series de tiempo

In [25]:
longitud = 12
tamanio_tanda = 32
generador = TimeseriesGenerator(datos_entreno_escalados,
                                datos_entreno_escalados,
                                length = longitud,
                                batch_size = tamanio_tanda)

In [26]:
X,y = generador[0]

In [27]:
print(f'Dado el arreglo: \n{X.flatten()}')
print(f'Predecir esta y: \n {y}')

Dado el arreglo: 
[0.00946788 0.06107842 0.11324343 0.14143325 0.18979603 0.30491086
 0.30149567 0.27411688 0.19733743 0.08531309 0.01561334 0.
 0.06107842 0.11324343 0.14143325 0.18979603 0.30491086 0.30149567
 0.27411688 0.19733743 0.08531309 0.01561334 0.         0.01773913
 0.11324343 0.14143325 0.18979603 0.30491086 0.30149567 0.27411688
 0.19733743 0.08531309 0.01561334 0.         0.01773913 0.09849259
 0.14143325 0.18979603 0.30491086 0.30149567 0.27411688 0.19733743
 0.08531309 0.01561334 0.         0.01773913 0.09849259 0.15587533
 0.18979603 0.30491086 0.30149567 0.27411688 0.19733743 0.08531309
 0.01561334 0.         0.01773913 0.09849259 0.15587533 0.18355852
 0.30491086 0.30149567 0.27411688 0.19733743 0.08531309 0.01561334
 0.         0.01773913 0.09849259 0.15587533 0.18355852 0.20875425
 0.30149567 0.27411688 0.19733743 0.08531309 0.01561334 0.
 0.01773913 0.09849259 0.15587533 0.18355852 0.20875425 0.37144257
 0.27411688 0.19733743 0.08531309 0.01561334 0.         0.01

## Construcción del modelo

Tenemos 1 feature.

In [28]:
n_features = 1

In [29]:
modelo = Sequential()
modelo.add(LSTM(100, activation = 'relu', input_shape = (longitud,
                                                         n_features)))
modelo.add(Dense(1))
modelo.compile(optimizer = 'adam', loss = 'mse')

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [30]:
modelo.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 100)            │        40,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,901 (159.77 KB)

 Trainable params: 40,901 (159.77 KB)

 Non-trainable params: 0 (0.00 B)

## Entranamiento:

In [31]:
detencion_temprana = EarlyStopping(monitor = 'val_loss', patience = 2)

In [32]:
generador_validacion = TimeseriesGenerator(datos_prueba_escalados,
                                           datos_prueba_escalados,
                                           length = longitud,
                                           batch_size = tamanio_tanda)

In [33]:
modelo.fit(generador, epochs = 20,
                    validation_data = generador_validacion,
                    callbacks = [detencion_temprana])

Epoch 1/20


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning:

Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.



19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.1383 - val_loss: 0.0106
Epoch 2/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0263 - val_loss: 0.0089
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0248 - val_loss: 0.0103
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0212 - val_loss: 0.0065
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0194 - val_loss: 0.0060
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0179 - val_loss: 0.0051
Epoch 7/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0144 - val_loss: 0.0046
Epoch 8/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0135 - val_loss: 0.0026
Epoch 9/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0109 - val_loss: 0.0024
Epoch 10/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0086 - val_loss: 0.0130
Epoch 11/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0183 - val_loss: 0.0072


In [34]:
perdidas = pd.DataFrame(modelo.history.history)

In [35]:
fig = px.line(perdidas)
fig.show()

## Evaluación

In [36]:
primera_tanda_eval = datos_entreno_escalados[-longitud:]

In [37]:
primera_tanda_eval = primera_tanda_eval.reshape((1, longitud, n_features))

In [38]:
modelo.predict(primera_tanda_eval)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step


array([[0.36533383]], dtype=float32)

In [39]:
datos_prueba_escalados[0]

array([0.48713795])

In [40]:
predicciones_prueba = []

primera_tanda_eval = datos_entreno_escalados[-longitud:]
tanda_actual = primera_tanda_eval.reshape((1, longitud, n_features))

for i in range(len(prueba)):
    prediccion_actual = modelo.predict(tanda_actual)[0]

    predicciones_prueba.append(prediccion_actual)

    tanda_actual = np.append(tanda_actual[:,1:,:],
                             [[prediccion_actual]], axis = 1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


In [41]:
predicciones_reales = scaler.inverse_transform(predicciones_prueba)

In [42]:
prueba['Predicciones'] = predicciones_reales

/tmp/ipython-input-1226055985.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [43]:
prueba

,IPN31152N,Predicciones
observation_date,,
2023-07-01,126.0569,109.251365
2023-08-01,122.6748,105.482910
2023-09-01,116.2177,102.028865
2023-10-01,109.3279,98.034473
2023-11-01,89.3353,95.575178
2023-12-01,85.2839,95.590775
2024-01-01,85.9573,97.014476
2024-02-01,110.0548,98.888963
2024-03-01,124.9787,99.675409


In [44]:
fig = px.line(prueba)
fig.show()

## Entrenar de nuevo

In [60]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

modelo = Sequential([
    LSTM(64, input_shape=(longitud, n_features), return_sequences=True),  # tanh por defecto
    Dropout(0.2),
    LSTM(32),
    Dense(1)
])
modelo.compile(optimizer=Adam(1e-3), loss='mse')

es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
modelo.fit(
    generador,
    validation_data=generador_validacion,
    epochs=100,
    callbacks=[es],
    verbose=0
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



#### PRONÓSTICO ROLLING SOBRE EL CONJUNTO DE PRUEBA


In [61]:
yhat_test_scaled = []
tanda = datos_entreno_escalados[-longitud:].copy()  # última ventana del set de entrenamiento

for _ in range(len(prueba)):
    x = tanda.reshape(1, longitud, n_features)
    yhat = modelo.predict(x, verbose=0)           # predice el siguiente paso (en escala MinMax)
    yhat_test_scaled.append(yhat[0])
    tanda = np.vstack([tanda[1:], yhat])          # avanzamos la ventana con la predicción

pronostico_test = scaler.inverse_transform(np.array(yhat_test_scaled))
prueba = prueba.copy()
prueba['Pronostico'] = pronostico_test


### PRONÓSTICO HACIA EL FUTURO (p pasos)

In [62]:
p = 36  # <- 36 meses en el futuro a predecir

# Escalamos TODO el histórico con el MISMO scaler (entrenado con 'entreno')
serie_escalada = scaler.transform(df.values)

tanda = serie_escalada[-longitud:]               # última ventana del histórico completo
tanda = tanda.reshape(1, longitud, n_features)

yhat_fut_scaled = []
for _ in range(p):
    yhat = modelo.predict(tanda, verbose=0)
    yhat_fut_scaled.append(yhat[0])
    tanda = np.append(tanda[:, 1:, :], yhat.reshape(1, 1, 1), axis=1)

yhat_fut = scaler.inverse_transform(np.array(yhat_fut_scaled))

# Índice correcto: mes siguiente al último dato real
inicio = df.index[-1] + pd.offsets.MonthBegin(1)
idx_fut = pd.date_range(start=inicio, periods=p, freq='MS')

df_pronostico = pd.DataFrame(yhat_fut, index=idx_fut, columns=['Pronostico'])


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



#### GRÁFICOS (real + pronóstico en prueba + pronóstico futuro)

In [63]:
import plotly.express as px
import plotly.graph_objects as go

fig = px.line(df, title="Serie original y pronósticos")
fig.add_scatter(x=prueba.index, y=prueba['Pronostico'].ravel(),
                name='Pronóstico (sobre prueba)')
fig.add_scatter(x=df_pronostico.index, y=df_pronostico['Pronostico'].ravel(),
                name='Pronóstico (futuro)')

fig.update_layout(
    width=850, height=450,
    xaxis_range=[df.index.min(), df_pronostico.index.max()],
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)
fig.show()


/tmp/ipython-input-4180248391.py:5: FutureWarning:

Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.

/tmp/ipython-input-4180248391.py:7: FutureWarning:

Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.



In [70]:
# para df histórico
df = df.copy()
df.index = pd.to_datetime(df.index)     # por si viene como string
df = df.sort_index()
df.index.name = "observation_date"      # solo el nombre del eje

# para df_pronostico (la serie de pronóstico)
df_pronostico = df_pronostico.copy()
df_pronostico.index = pd.to_datetime(df_pronostico.index)
df_pronostico = df_pronostico.sort_index()
df_pronostico.index.name = "observation_date"

import plotly.express as px

# px.line en modo "wide": usa el índice como eje X automáticamente
fig = px.line(df)                       # histórico
fig2 = px.line(df_pronostico)           # pronóstico

# Forzar color naranja al pronóstico
for tr in fig2.data:
    tr.update(line=dict(color="orange"))

# Añadir la(s) traza(s) del pronóstico al gráfico base
fig.add_traces(list(fig2.data))

# Rango de X: del inicio histórico al final del pronóstico
xmin = df.index.min()
xmax = max(df.index.max(), df_pronostico.index.max())

fig.update_layout(
    width=800, height=420,
    xaxis_title="observation_date", yaxis_title="value",
    xaxis_range=[xmin, xmax],
    legend_title_text="variable",
    margin=dict(l=40, r=20, t=60, b=40)
)

fig.show()

### Analisis y discusión

Como podemos ver los scalers son muy importantes con las RNN, porque gracias a haber escalado los datos bien las predicciones de futuro si bien se logran captar de manera acertada, no picos y cambios abruptos no se logran de la manera correcta por el hecho de que subestima medias y prefiere hacer estimaciones más "suaves".

Conclusión
El early stoping que utilizamos si que nos ayudo a mejorar tanto el tiempo como predicciones ya que si no se aplicaba de manera correcta nos podía dar un sobre ajuste asi como también hacer un early demasiado temprano nos quita precisión en predicciones. El uso de scaler fue crucial para ajustar los datos y acercarnos a las predicciones generales, más no logra captar la volatibilidad de toda serie de tiempo ya que esto tambien necesita de un training mucho más exaustivo y obviamente que las predicciones no pueden captar totalmente la naturaleza de los datos reales